### Import packages

In [1]:
from __future__ import division, print_function
import glob
import os
import warnings
warnings.simplefilter("ignore")
import cv2
import PIL
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from Utils import L2Norm, cv2_scale, np_reshape

from numpy.linalg import norm
from scipy.optimize import linear_sum_assignment


### Check GPU availability, using nvidia-smi 

In [2]:
# Since there are two GPUs on each pelican server, you can either select it as 0 or 1
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

print(torch.__version__)
print(torch.version.cuda)  
print(torch.cuda.is_available())

1.8.1+cu101
10.1
True


---
---
<a id='load_data_cell'></a>
## Load and visualize the data

In this section, we will 
#### 1. [Define a PyTorch dataset](#pytorch_dataset_cell)
#### 2. [Define a PyTorch dataloader](#pytorch_dataloader_cell)
#### 3. [Load data](#load_dataset_cell)
#### 4. [Visualizaiton of the Training and Testing Data](#visualize_dataset_cell)

[BackToTop](#top_cell)

---

<a id='pytorch_dataset_cell'></a>
### Define PyTorch dataset

[BackToSection](#load_data_cell)

In [3]:
class TripletPhotoTour(dset.PhotoTour):
    """
    From the PhotoTour Dataset it generates triplet samples
    note: a triplet is composed by a pair of matching images and one of
    different class.
    """
    def __init__(self, train=True, transform=None, batch_size = None,load_random_triplets = False,  *arg, **kw):
        super(TripletPhotoTour, self).__init__(*arg, **kw)
        self.transform = transform
        self.out_triplets = load_random_triplets
        self.train = train
        self.n_triplets = args.n_triplets
        
        
        self.batch_size = batch_size
        self.triplets = self.generate_triplets(self.labels, self.n_triplets)
            
    @staticmethod
    def generate_triplets(labels, num_triplets):
        def create_indices(_labels):
            inds = dict()
            for idx, ind in enumerate(_labels):
                if ind not in inds:
                    inds[ind] = []
                inds[ind].append(idx)
            return inds

        triplets = []
        indices = create_indices(labels.numpy())
        unique_labels = np.unique(labels.numpy())
        n_classes = unique_labels.shape[0]
        # add only unique indices in batch
        already_idxs = set()

        for x in tqdm(range(num_triplets)):
            if len(already_idxs) >= args.batch_size:
                already_idxs = set()
            c1 = np.random.randint(0, n_classes)
            while c1 in already_idxs:
                c1 = np.random.randint(0, n_classes)
            already_idxs.add(c1)
            c2 = np.random.randint(0, n_classes)
            while c1 == c2:
                c2 = np.random.randint(0, n_classes)
            if len(indices[c1]) == 2:  # hack to speed up process
                n1, n2 = 0, 1
            else:
                n1 = np.random.randint(0, len(indices[c1]))
                n2 = np.random.randint(0, len(indices[c1]))
                while n1 == n2:
                    n2 = np.random.randint(0, len(indices[c1]))
            n3 = np.random.randint(0, len(indices[c2]))
            triplets.append([indices[c1][n1], indices[c1][n2], indices[c2][n3]])
        return torch.LongTensor(np.array(triplets))

    def __getitem__(self, index):
        def transform_img(img):
            if self.transform is not None:
                img = self.transform(img.numpy())
            return img

        t = self.triplets[index]
        a, p, n = self.data[t[0]], self.data[t[1]], self.data[t[2]]

        img_a = transform_img(a)
        img_p = transform_img(p)
        img_n = None
        if self.out_triplets:
            img_n = transform_img(n)
        # transform images if required
        if args.fliprot:
            do_flip = random.random() > 0.5
            do_rot = random.random() > 0.5
            if do_rot:
                img_a = img_a.permute(0,2,1)
                img_p = img_p.permute(0,2,1)
                if self.out_triplets:
                    img_n = img_n.permute(0,2,1)
            if do_flip:
                img_a = torch.from_numpy(deepcopy(img_a.numpy()[:,:,::-1]))
                img_p = torch.from_numpy(deepcopy(img_p.numpy()[:,:,::-1]))
                if self.out_triplets:
                    img_n = torch.from_numpy(deepcopy(img_n.numpy()[:,:,::-1]))
        return (img_a, img_p, img_n)
        

    def __len__(self):
        return self.triplets.size(0)
        

<a id='pytorch_dataloader_cell'></a>
### Define the dataloader
[BackToSection](#load_data_cell)

In [4]:
def create_loaders(dataset_names, load_random_triplets = False, verbose=False):
    """
    For training, we use dataset 'liberty';
    For testing, we use dataset 'notredame' and 'yosemite'
    
    """
    test_dataset_names = copy(dataset_names)
    test_dataset_names.remove(args.training_set)

    kwargs = {'num_workers': args.num_workers, 'pin_memory': args.pin_memory} if args.cuda else {}

    np_reshape64 = lambda x: np.reshape(x, (64, 64, 1))
    transform_test = transforms.Compose([
            transforms.Lambda(np_reshape64),
            transforms.ToPILImage(),
            transforms.Resize(32),
            transforms.ToTensor()])
    transform_train = transforms.Compose([
            transforms.Lambda(np_reshape64),
            transforms.ToPILImage(),
            transforms.RandomRotation(5,PIL.Image.BILINEAR),
            transforms.RandomResizedCrop(32, scale = (0.9,1.0),ratio = (0.9,1.1)),
            transforms.Resize(32),
            transforms.ToTensor()])
    transform = transforms.Compose([
            transforms.Lambda(cv2_scale),
            transforms.Lambda(np_reshape),
            transforms.ToTensor(),
            transforms.Normalize((args.mean_image,), (args.std_image,))])
    if not args.augmentation:
        transform_train = transform
        transform_test = transform
    train_loader = torch.utils.data.DataLoader(
            TripletPhotoTour(train=True,
                             load_random_triplets = load_random_triplets,
                             batch_size=args.batch_size,
                             root=args.dataroot,
                             name=args.training_set,
                             download=True,
                             transform=transform_train),
                             batch_size=args.batch_size,
                             shuffle=False, **kwargs)

    test_loaders = [{'name': name,
                     'dataloader': torch.utils.data.DataLoader(
             TripletPhotoTour(train=False,
                     batch_size=args.test_batch_size,
                     load_random_triplets = load_random_triplets,
                     root=args.dataroot,
                     name=name,
                     download=True,
                     transform=transform_test),
                        batch_size=args.test_batch_size,
                        shuffle=False, **kwargs)}
                    for name in test_dataset_names]

    return train_loader, test_loaders[0]

---
<a id='build_network_cell'></a>
## Build an exmaple deep network  

In this section, we will:
#### 1. [Build the deep network: DesNet](#build_desNet_cell)
#### 2. [Setup optimization](#set_opt_cell)

[BackToTop](#top_cell)

---

<a id='build_desNet_cell'></a>
### Build the deep network: DesNet
The DesNet is a simple CNN network, which only contains two CNN blocks.


[BackToSection](#build_network_cell)

In [24]:
# load network from the python file. You need to submit these .py files to TA
from CNN1 import DesNet       # uncomment this line if you are using DesNet from CNN1.py
#from CNN2 import DesNet      # uncomment this line if you are using DesNet from CNN2.py
#from CNN3 import DesNet
#from class_CNN3 import DesNet      # uncomment this line if you are using DesNet from CNN3.py

model = DesNet()
# check model architecture

print(model)

if args.cuda:
    model.cuda()

DesNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Conv2d(128, 128, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
)


<a id='set_opt_cell'></a>
### Define optimization
We will use SGD, but you can change it to ADAM by modifying arg.lr in config_profile.py

[BackToSection](#build_network_cell)

In [25]:
# define optimizer
def create_optimizer(model, new_lr):
    # setup optimizer
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=new_lr,
                              momentum=0.9, dampening=0.9,
                              weight_decay=args.wd)
    else:
        raise Exception('Not supported optimizer: {0}'.format(args.optimizer))
    return optimizer
optimizer1 = create_optimizer(model.features, args.lr)

---
<a id='generate_deep_features_cell'></a>
## Generate deep features
In this section, we will use your trained network to generate deep features for each patch:
#### 1. [load weights](#load_weights_module_cell)
#### 2. [load patches](#load_raw_patch_files_module_cell)
#### 3. [get deep features](#get_deep_features_module_cell)



[BackToTop](#top_cell)

<a id='load_weights_module_cell'></a>
### Load network weights
[BackToSection](#generate_deep_features_cell)

In [27]:
trained_weight_path = "models/liberty_train/_liberty_min_as_fliprot/CNN1.pth" # suppose you select  checkpoint_4.pth as the best model for this architecture
#trained_weight_path = "class_checkpoint.pth"
test_model = DesNet()
if args.cuda:
    test_model.cuda()
trained_weight = torch.load(trained_weight_path)['state_dict']
test_model.load_state_dict(trained_weight)
test_model.eval()

DesNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Conv2d(128, 128, kernel_size=(8, 8), stride=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
)

<a id='load_raw_patch_files_module_cell'></a>
### Load raw patch files
Assume that the raw patch file is stored as patches.pth

[BackToSection](#generate_deep_features_cell)

In [27]:
patches_dir = "cd ../My_Results/Query_Images_Matching/Images_patches.pth"            # these patches are from keypoint detection results
patches = torch.load(patches_dir)

print(patches.shape)                  # in your case, the shape should be [10, 200, 1, 32, 32]
num_imgs, num_pts, _, _, _ = patches.shape
patches = patches.view(-1, 1, 32, 32).cuda()
print(patches.shape)


torch.Size([136, 20, 1, 32, 32])
torch.Size([2720, 1, 32, 32])


<a id='get_deep_features_module_cell'></a>
### Get deep features
[BackToSection](#generate_deep_features_cell)

In [28]:
features = test_model(patches)
print(features.shape)
features = features.view(num_imgs, num_pts, 128).cpu().data
print(features.shape)                  # in your case, the shape should be [10, 200, 128]

torch.Size([2720, 128])
torch.Size([136, 20, 128])


In [29]:
# save to file, with the name of *_features_CNN*.pth
features_dir = "cd ../My_Results/Query_Images_Matching/Images_features_CNN1.pth"
#features_dir = "cd ../My_Results/Query_Images_Matching/Images_features_class_CNN3.pth"
torch.save(features, features_dir)

In [28]:
patches_dir = "cd ../My_Results/Query_Images_Matching/Images_features_CNN1.pth" 
#patches_dir = "cd ../My_Results/Query_Images_Matching/Images_features_class_CNN3.pth"            # these patches are from keypoint detection results
features_Images = torch.load(patches_dir)

patches_dir = "cd ../My_Results/Query_Images_Matching/Query_features_CNN1.pth" 
#patches_dir = "cd ../My_Results/Query_Images_Matching/Query_features_class_CNN3.pth" 
features_Query = torch.load(patches_dir)

print(features_Images.shape)
print(features_Query.shape)

torch.Size([136, 20, 128])
torch.Size([34, 20, 128])


In [2]:
#features_Images
#features_Query

def manytomany(features_Query, features_Images, threshold):
    sqr = np.zeros((34,136))
    for num_Q in range(len(features_Query)):
        for num_I in range(len(features_Images)):         
            skl = []
            x = 0
            for Q_keypoint in range(20):
                for I_keypoint in range(20):                        
                        #similarity = 0.5 *( 1 + 
                        #            ( features_Query[num_Q][Q_keypoint].dot(features_Images[num_I][I_keypoint]) ) / 
                        #            ( norm(features_Query[num_Q][Q_keypoint],2) * norm(features_Images[num_I][I_keypoint],2) ) )
                        similarity = F.cosine_similarity(features_Query[num_Q][Q_keypoint], features_Images[num_I][I_keypoint], dim=0)
                        skl.append(similarity)
            x = skl / norm(skl,2)
            # check the threshold
            for x_idx in range(20 * 20):
                if x[x_idx] > threshold:
                    x[x_idx] = 1
                else:
                    x[x_idx] = 0
            # skl.dot(x) is the sum of similarity of matching from 1 query to 1 image
            skl = np.array(skl)
            x = np.array(x)
            
            sqr[num_Q][num_I] = skl.dot(x)
    return sqr

def get_indicator_matrix(cost_matrix):
    x = np.zeros((20,20))
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    for i in range(20):
        x[i][col_ind[i]] = 1
    return x
    
    
    #trace = cost_matrix[row_ind, col_ind].sum()   
    #return trace

def onetoone(features_Query, features_Images):
    sqr = np.zeros((34,136))
    for num_Q in range(len(features_Query)):
        for num_I in range(len(features_Images)):
            ckl = np.zeros((20,20))
            for Q_keypoint in range(20):
                for I_keypoint in range(20):
                        #dist = 1 - 0.5 *( 1 + 
                        #        ( features_Query[num_Q][Q_keypoint].dot(features_Images[num_I][I_keypoint]) ) / 
                        #        ( norm(features_Query[num_Q][Q_keypoint],2) * norm(features_Images[num_I][I_keypoint],2) ) )
                        dist = 1 - F.cosine_similarity(features_Query[num_Q][Q_keypoint], features_Images[num_I][I_keypoint], dim=0)
                        ckl[Q_keypoint][I_keypoint] = dist
            indicator_matrix = get_indicator_matrix(ckl)
            skl = 1 - ckl
            diagonal_value = (skl.transpose()).dot(indicator_matrix)
            max_trace = 0
            for i in range(20):
                max_trace = max_trace + diagonal_value[i][i]
            
            #sqr.append(min_trace_value)
            sqr[num_Q][num_I] = max_trace
    return sqr 
                    

In [30]:
My_CNN_one_to_one_match = onetoone(features_Query, features_Images)
My_CNN_many_to_many_match = manytomany(features_Query,features_Images, 0.072)

In [15]:
Class_CNN_one_to_one_match = onetoone(features_Query, features_Images)
Class_CNN_many_to_many_match = manytomany(features_Query,features_Images, 0.072)

In [17]:
#Run this block after got the above four value.
final_retrieval = []
for j in range(4):    
    if j == 0:
        test_tensor = torch.Tensor(My_CNN_many_to_many_match)
    if j == 1:
        test_tensor = torch.Tensor(My_CNN_one_to_one_match)
    if j == 2:
        test_tensor = torch.Tensor(Class_CNN_many_to_many_match)
    if j == 3:
        test_tensor = torch.Tensor(Class_CNN_one_to_one_match)        
    retrieval_matrix = torch.Tensor(np.zeros((34,4)))
    for i in range(34):
        R = torch.topk(test_tensor[i], 4, dim=0, largest=True, sorted=True, out=None)
        retrieval_matrix[i] = R[1]
    final_retrieval.append(retrieval_matrix)    
all_retrieval_matrix = torch.stack(final_retrieval, dim=0)
print(all_retrieval_matrix.shape)

retrieval_pth = "cd ../My_Results/Query_Images_Matching/retrieval.pth" 
torch.save(all_retrieval_matrix,retrieval_pth)

torch.Size([4, 34, 4])


In [33]:
def Precision (test_method, k):
    precision_list = []
    test_tensor = torch.Tensor(test_method)
    for query_num in range(34):
        #print(query_num)
        true_positive = 0
        Top_K_idx = torch.topk(test_tensor[query_num], k, dim=0, largest=True, sorted=True, out=None)
        for k_idx in range(k):
            #print(Top_K_idx[1][k_idx])
            if (Top_K_idx[1][k_idx] == 4 * query_num) or (Top_K_idx[1][k_idx] == 4 * query_num + 1) or (Top_K_idx[1][k_idx] == 4 * query_num + 2) or (Top_K_idx[1][k_idx] == 4 * query_num + 3):
                true_positive = true_positive + 1
        precision_list.append(true_positive / k)
    precision_value = sum(precision_list) / 34
    return precision_value
#print(precision_value)

def Recall (test_method, k):
    recall_list = []
    test_tensor = torch.Tensor(test_method)
    for query_num in range(34):
        #print(query_num)
        true_positive = 0
        Top_K_idx = torch.topk(test_tensor[query_num], k, dim=0, largest=True, sorted=True, out=None)
        for k_idx in range(k):
            #print(Top_K_idx[1][k_idx])
            if (Top_K_idx[1][k_idx] == 4 * query_num) or (Top_K_idx[1][k_idx] == 4 * query_num + 1) or (Top_K_idx[1][k_idx] == 4 * query_num + 2) or (Top_K_idx[1][k_idx] == 4 * query_num + 3):
                true_positive = true_positive + 1
        recall_list.append(true_positive / 4) # 4 is the ground truth in each image
    recall_value = sum(recall_list) / 34
    return recall_value

In [32]:
T = torch.cat((features_Query, features_Images))
print(T.shape)
filename = "cd ../My_Results/Query_Images_Matching/features1.pth" #feature1 is my CNN, feature2 is class CNN
torch.save(T,filename)

torch.Size([170, 20, 128])
